In [1]:
library(Matrix)
library(RBGL)
library(snow)
library(graph)
library(randomForest)

Loading required package: graph

Loading required package: BiocGenerics


Attaching package: ‘BiocGenerics’


The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


The following objects are masked from ‘package:base’:

    Filter, Find, Map, Position, Reduce, anyDuplicated, append,
    as.data.frame, basename, cbind, colnames, dirname, do.call,
    duplicated, eval, evalq, get, grep, grepl, intersect, is.unsorted,
    lapply, mapply, match, mget, order, paste, pmax, pmax.int, pmin,
    pmin.int, rank, rbind, rownames, sapply, setdiff, sort, table,
    tapply, union, unique, unsplit, which.max, which.min


randomForest 4.7-1

Type rfNews() to see new features/changes/bug fixes.


Attaching package: ‘randomForest’


The following object is masked from ‘package:BiocGenerics’:

    combine




In [2]:
# Set maximum request time
getOption('timeout')
options(timeout=100)

[1] 60

# Build a network graph from STRING database


In [3]:
# Preprocess STRING DATA to generate a network graph
build_string_network <- function(data) {
    # downloading data if it does not exist in the directory
    if(!file.exists(data)){
        download.file(url = "http://version10.string-db.org/download/protein.links.v10/9606.protein.links.v10.txt.gz",
                      destfile = data)
    }
    
    # Reading STRING data
    data = read.table(data, stringsAsFactors=F, header=T)
    # Cleaning up protein IDs
    data[,1] = gsub("9606." , "", data[,1], fixed=T)
    data[,2] = gsub("9606.", "", data[,2], fixed=T)
    # Remove interactions <400
    threshold_data = data[data[,3]>400,]
    # Build a network graph from the STRING data
    string_network = ftM2graphNEL(as.matrix(threshold_data[,1:2]))
    # Calculating shortest paths between the protein IDs
    shortest_path = johnson.all.pairs.sp(string_network)
    save(string_network, shortest_path, file="./rData/STRING_network.Rdata")
    return(shortest_path)
}

In [4]:
shortest_path = build_string_network("./data/9606.protein.links.v10.txt.gz")

# Build a genewise expression level score matrix from BrainSpan data


In [6]:
# Preprocess BRAINSPAN DATA to generate a genewise expression matrix
build_brainspan_matrix <- function(data) {  
    # downloading data if it does not exist in the directory
    if(!file.exists(data)){
        download.file(url = "http://www.brainspan.org/api/v2/well_known_file_download/267666529",
                  destfile = data)
        unzip(data, exdir = "./data/brainspan/")
        }
    # Reading BRAINSPAN data
    expr_matrix = read.table("./data/brainspan/expression_matrix.csv",
                   sep=",",header=F,stringsAsFactors=F)
    annotations = read.table("./data/brainspan/rows_metadata.csv",
                     sep=",",header=T,stringsAsFactors=F)
    fac = read.table("./data/brainspan/columns_metadata.csv",
                     sep=",",header=T,stringsAsFactors=F)
    expr_matrix = as.matrix(expr_matrix[,-1])
    
    # Using Gene symbols to map missing entrez gene IDs
    entrezToSymbol = read.table("data/entrezgene2symbol.csv", sep=",", header = T, stringsAsFactors = F)
    entrezToSymbol = structure(entrezToSymbol$symbol, names = entrezToSymbol$entrez)
    annotations[is.na(annotations$entrez_id),"entrez_id"] = names(entrezToSymbol)[match(annotations[is.na(annotations$entrez_id),]$gene_symbol,entrezToSymbol)]
    rownames(expr_matrix) = annotations$entrez
    
    # Convert all ages into weeks
    age_col = fac$age
    all_age_data = strsplit(age_col,split=" ")
    ageInWeeks = sapply(all_age_data,function(age) {
      if(age[2]=="pcw") weeks=as.numeric(age[1])*1
      if(age[2]=="mos") weeks=as.numeric(age[1])*4.33 + 38
      if(age[2]=="yrs") weeks=as.numeric(age[1])*52 + 38
      return(weeks)
    }
    )
    
    # Filter for structures found in most samples
    strucs = fac$structure_acronym
    most_sample_str = names(table(strucs)[table(strucs)>20])


    # Lower number of parallel cores if < 4 available
    cluster_obj = makeSOCKcluster(4)

    # Smoothing and interpolation of the data
    smooth = function(y,x_age){
      lw = lowess(log(x_age),y,f=1/3)
      approx_est = approx(lw ,
                   xout = seq(2, log(2118), length.out = 50),
                   rule = 2)
      return(approx_est$y)
    }
    
    #initializing expression matrix where the elements represent expression in a brain region
    # Rows: timepoints
    # Columns: Genes
    expr_m = list()

    print("Generting Expression Matrix...")
    for(idx in 1:length(most_sample_str)){
      curr = most_sample_str[idx]
      X = expr_matrix[,strucs==curr]
      x_age = ageInWeeks[strucs==curr]
      expr_m[[idx]] = parApply(cluster_obj,X,1,smooth,x_age)
      names(expr_m)[idx] = most_sample_str[idx]
      print(idx)
    }

    # Perform transpose of the matrix
    expr_m = lapply(expr_m,t)

    # row-bind same genes together to ensure the each gene has only one matrix where
    # Rows: Brain regions
    # Columns: timepoints
    expr_m = parLapply(cluster_obj, 1:nrow(expr_m[[1]]), function(idx, y){
      do.call('rbind', lapply(y, function(x) {
        x[idx,]
      }))
    }, expr_m)

    # Relabel matrix rows by entrez IDs
    names(expr_m) = rownames(expr_matrix)
    new_rows = rownames(expr_m[[1]])

    # Scaling the expression matrix
    scale_func = function(val){
      matrix(
        scale(as.numeric(val)),
        nrow = 16,
        ncol = 50,
        dimnames = list( rownames(val) , colnames(val) )
      )
      }

    scaled_expr_m = lapply(expr_m,scale_func)
    
    # Map entrez ID to the ensembl protein ID
    enzToEns = read.table("data/entrez_gene_id.vs.string.v10.28042015.tsv",sep="\t",stringsAsFactors=F)
   
    # Clean up protein names
    enzToEns[,2] = gsub("9606.","",enzToEns[,2],fixed=T)
    enzToEns = structure(enzToEns[,2],names=as.character(enzToEns[,1]))

    # Vectorizing the scaled gene-wise expression matricies to ensure rows represent genes
    genewise_expr = sapply(scaled_expr_m,as.numeric)
    genewise_expr = t(genewise_expr)
    genewise_expr = genewise_expr[!is.na(rownames(genewise_expr)),]

    # Map the missing IDs in the BRAINSPAN data
    missing_ids = read.table("data/brainspan_missing_ids.txt",sep="\t",header=T,stringsAsFactors=F)
    missing_ids = missing_ids[missing_ids$Protein.stable.ID%in%rownames(shortest_path),]
    missing_ids = missing_ids[!duplicated(missing_ids$NCBI.gene.ID),]
    missing_ids = structure(missing_ids$Protein.stable.ID,names=as.character(missing_ids$NCBI.gene.ID))
    enzToEns = c(enzToEns,missing_ids)
    rownames(genewise_expr) = enzToEns[rownames(genewise_expr)]
    colnames(genewise_expr) = 1:800

    save(entrezToSymbol, missing_ids, enzToEns, file = "rData/id_conversion.Rdata")

    # Handling missing values by replacing with Median
    genewise_expr = na.roughfix(genewise_expr)
    save(genewise_expr, file="rData/Genewise_expr_matrix.Rdata")
    return(genewise_expr)
    
    
}

In [7]:
genewise_expr = build_brainspan_matrix("./data/genes_matrix_csv.zip")

[1] "Generting Expression Matrix..."
[1] 1
[1] 2
[1] 3
[1] 4
[1] 5
[1] 6
[1] 7
[1] 8
[1] 9
[1] 10
[1] 11
[1] 12
[1] 13
[1] 14
[1] 15
[1] 16


# Define Training Features

In [8]:
# Getting Training Labels
gen_training_labels <- function(shortest_path, genewise_expr) { 

    # Read SFARI genes and gene IDs data
    sfari = read.table("data/SFARI-Gene_genes_export01-11-2017.csv",sep=",",header=T,stringsAsFactors=F)
    sfari_ids = read.table("data/sfari_gene_ids.txt",sep="\t",stringsAsFactors=F,header=T)

    # Top Genes contains SFARI 1 and 2 genes
    top_genes = sfari$gene.symbol[sfari$gene.score %in% c("1","2")]
    top_genes = sfari_ids$Protein.stable.ID[sfari_ids$Gene.name %in% top_genes]
    top_genes = unique( top_genes[ top_genes %in% rownames(shortest_path) ] )
    # Positive gene examples contains top genes that are also present in STRING
    pos_genes = rownames(shortest_path)[rownames(shortest_path) %in% top_genes ]
    pos_genes = unique(pos_genes)
    pos_genes = pos_genes[ !is.na(pos_genes) ]
    labels = rownames(shortest_path)[ rownames(shortest_path) %in% rownames(genewise_expr) ]

    # Negative gene examples are set as random background genes
    set.seed(3716359)
    neg_genes = sample(labels[ !(labels %in% sfari_ids$Protein.stable.ID) ] , 1000)

    save(pos_genes, neg_genes, file="rData/training_labels.Rdata")

}


In [9]:
gen_training_labels(shortest_path, genewise_expr)